<a href="https://colab.research.google.com/github/DigitalEdgeAi/BOT/blob/main/Basic_Python_Structure_for_Event_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic Python Structure for a Specialized Event/Workshop Platform

# --- Data Structures (Simplified) ---

class Event:
    """Represents a single event or workshop."""
    def __init__(self, event_id, title, description, date, time, location, price, organizer_id, capacity):
        self.event_id = event_id
        self.title = title
        self.description = description
        self.date = date # Using string for simplicity, could use datetime objects
        self.time = time # Using string for simplicity
        self.location = location
        self.price = price
        self.organizer_id = organizer_id
        self.capacity = capacity
        self.attendees = [] # List of user_ids who have booked

    def add_attendee(self, user_id):
        """Adds a user to the event's attendees list if capacity allows."""
        if len(self.attendees) < self.capacity:
            self.attendees.append(user_id)
            return True
        return False # Event is full

    def remove_attendee(self, user_id):
        """Removes a user from the event's attendees list."""
        if user_id in self.attendees:
            self.attendees.remove(user_id)
            return True
        return False

class User:
    """Represents a platform user (attendee or organizer)."""
    def __init__(self, user_id, name, email, user_type): # user_type could be 'attendee' or 'organizer'
        self.user_id = user_id
        self.name = name
        self.email = email
        self.user_type = user_type
        self.registered_events = [] # List of event_ids the user is attending

    def register_for_event(self, event_id):
        """Registers the user for an event."""
        if event_id not in self.registered_events:
            self.registered_events.append(event_id)
            return True
        return False

    def unregister_from_event(self, event_id):
        """Unregisters the user from an event."""
        if event_id in self.registered_events:
            self.registered_events.remove(event_id)
            return True
        return False

# --- Core Platform Logic (Simplified) ---

class EventPlatform:
    """Manages users and events on the platform."""
    def __init__(self):
        self.users = {} # Dictionary to store users: {user_id: User object}
        self.events = {} # Dictionary to store events: {event_id: Event object}
        self._next_user_id = 1
        self._next_event_id = 1

    def create_user(self, name, email, user_type):
        """Creates a new user."""
        user_id = self._next_user_id
        new_user = User(user_id, name, email, user_type)
        self.users[user_id] = new_user
        self._next_user_id += 1
        print(f"User created: {new_user.name} ({new_user.user_type}) with ID {user_id}")
        return new_user

    def create_event(self, title, description, date, time, location, price, organizer_id, capacity):
        """Creates a new event."""
        if organizer_id not in self.users or self.users[organizer_id].user_type != 'organizer':
            print(f"Error: Organizer with ID {organizer_id} not found or is not an organizer.")
            return None

        event_id = self._next_event_id
        new_event = Event(event_id, title, description, date, time, location, price, organizer_id, capacity)
        self.events[event_id] = new_event
        self._next_event_id += 1
        print(f"Event created: '{new_event.title}' with ID {event_id}")
        return new_event

    def find_events(self, query=None, date=None, location=None):
        """Searches for events based on criteria."""
        results = []
        for event in self.events.values():
            match = True
            if query and query.lower() not in event.title.lower() and query.lower() not in event.description.lower():
                match = False
            if date and event.date != date:
                match = False
            if location and location.lower() not in event.location.lower():
                match = False
            if match:
                results.append(event)
        return results

    def book_event(self, user_id, event_id):
        """Handles the booking process for a user and event."""
        user = self.users.get(user_id)
        event = self.events.get(event_id)

        if not user:
            print(f"Error: User with ID {user_id} not found.")
            return False
        if not event:
            print(f"Error: Event with ID {event_id} not found.")
            return False

        if event.add_attendee(user_id):
            user.register_for_event(event_id)
            print(f"User {user.name} successfully booked '{event.title}'.")
            # Here you would integrate payment processing and fee calculation
            # For simplicity, we'll just print a message
            print(f"Processing payment for {event.price} (plus service fee).")
            return True
        else:
            print(f"Booking failed for '{event.title}': Event is full.")
            return False

    def cancel_booking(self, user_id, event_id):
        """Handles booking cancellation."""
        user = self.users.get(user_id)
        event = self.events.get(event_id)

        if not user or not event:
            print("Error: User or Event not found.")
            return False

        if user_id in event.attendees:
            event.remove_attendee(user_id)
            user.unregister_from_event(event_id)
            print(f"User {user.name} successfully cancelled booking for '{event.title}'.")
            # Here you would handle refunds
            return True
        else:
            print(f"Cancellation failed: User {user.name} was not booked for '{event.title}'.")
            return False

# --- Example Usage ---

if __name__ == "__main__":
    platform = EventPlatform()

    # Create users
    organizer1 = platform.create_user("Art Studio London", "artstudio@example.com", "organizer")
    attendee1 = platform.create_user("Alice Smith", "alice@example.com", "attendee")
    attendee2 = platform.create_user("Bob Johnson", "bob@example.com", "attendee")

    # Create events
    painting_workshop = platform.create_event(
        "Beginner's Oil Painting",
        "Learn the basics of oil painting techniques.",
        "2025-05-10",
        "14:00",
        "Art Studio London, Central London",
        50.00,
        organizer1.user_id,
        10
    )

    cooking_class = platform.create_event(
        "Italian Pasta Making",
        "Master the art of fresh pasta.",
        "2025-05-17",
        "18:30",
        "The Culinary School, West London",
        75.00,
        organizer1.user_id, # Assuming the same organizer for simplicity
        8
    )

    # Search for events
    print("\nSearching for 'painting' events:")
    painting_results = platform.find_events(query="painting")
    for event in painting_results:
        print(f"- {event.title} on {event.date} at {event.location}")

    print("\nSearching for events on 2025-05-17:")
    date_results = platform.find_events(date="2025-05-17")
    for event in date_results:
        print(f"- {event.title} on {event.date} at {event.location}")

    # Book events
    platform.book_event(attendee1.user_id, painting_workshop.event_id)
    platform.book_event(attendee2.user_id, painting_workshop.event_id)
    platform.book_event(attendee1.user_id, cooking_class.event_id)

    # Attempt to book a full event (if capacity was low)
    # platform.book_event(attendee3.user_id, painting_workshop.event_id) # Example if another user tried to book

    # View attendees for an event
    print(f"\nAttendees for '{painting_workshop.title}':")
    for user_id in painting_workshop.attendees:
        user = platform.users[user_id]
        print(f"- {user.name}")

    # View registered events for a user
    print(f"\nEvents booked by {attendee1.name}:")
    for event_id in attendee1.registered_events:
        event = platform.events[event_id]
        print(f"- {event.title}")

    # Cancel a booking
    platform.cancel_booking(attendee2.user_id, painting_workshop.event_id)

    # View attendees after cancellation
    print(f"\nAttendees for '{painting_workshop.title}' after cancellation:")
    for user_id in painting_workshop.attendees:
        user = platform.users[user_id]
        print(f"- {user.name}")

User created: Art Studio London (organizer) with ID 1
User created: Alice Smith (attendee) with ID 2
User created: Bob Johnson (attendee) with ID 3
Event created: 'Beginner's Oil Painting' with ID 1
Event created: 'Italian Pasta Making' with ID 2

Searching for 'painting' events:
- Beginner's Oil Painting on 2025-05-10 at Art Studio London, Central London

Searching for events on 2025-05-17:
- Italian Pasta Making on 2025-05-17 at The Culinary School, West London
User Alice Smith successfully booked 'Beginner's Oil Painting'.
Processing payment for 50.0 (plus service fee).
User Bob Johnson successfully booked 'Beginner's Oil Painting'.
Processing payment for 50.0 (plus service fee).
User Alice Smith successfully booked 'Italian Pasta Making'.
Processing payment for 75.0 (plus service fee).

Attendees for 'Beginner's Oil Painting':
- Alice Smith
- Bob Johnson

Events booked by Alice Smith:
- Beginner's Oil Painting
- Italian Pasta Making
User Bob Johnson successfully cancelled booking f